In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
##load dataset
file = "data"

train_info_path = file + "/info_train.csv"
train_work_path = file + "/dat_fill_work_data/new_work_train.csv"
train_label_path = file + "/label_train.csv"

test_info_path = file + "/info_test.csv"
test_work_path = file + "/dat_fill_work_data/new_work_test.csv"
test_label_path = file + "/label_test.csv"


## data frame
train_info = pd.read_csv(train_info_path)
train_work = pd.read_csv(train_work_path)
train_label = pd.read_csv(train_label_path)

test_info = pd.read_csv(test_info_path)
test_work = pd.read_csv(test_work_path)
test_label = pd.read_csv(test_label_path)

In [4]:
train_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247559 entries, 0 to 247558
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  247559 non-null  int64  
 1   id_bh               247559 non-null  int64  
 2   id_management       247559 non-null  int64  
 3   id_office           247559 non-null  object 
 4   company_type        247559 non-null  int64  
 5   job/role            247552 non-null  object 
 6   from_date           247559 non-null  int64  
 7   to_date             247559 non-null  int64  
 8   employee_lv         247559 non-null  float64
 9   address             247559 non-null  object 
 10  year_from_date      247559 non-null  int64  
 11  year_to_date        247559 non-null  int64  
 12  month_from_date     247559 non-null  int64  
 13  month_to_date       247559 non-null  int64  
 14  num_month_contract  247559 non-null  int64  
 15  num_year_contract   247559 non-nul

In [5]:
## concat 
work = pd.concat([train_work, test_work],axis=0)

## 1. Address

In [6]:
## format text
def preprocess_address(add):
    add = re.sub(r'(SÀI GÒN)|(TPHCM)|(HCM)|(TP HCM)','TP HỒ CHÍ MINH',add) #Change some symnonyms
    add = re.sub(r'(HN)','HÀ NỘI',add)
    add = re.sub(r'(BG)','BẮC GIANG',add)
    # Remove some special characters
    add = re.sub(r'([\;\:\|•«\n])', ' ', add)
    return add
## upper word
work['address'] = work['address'].fillna(-99)
work['address'] = work.apply(lambda row: str(row.address).upper(), axis = 1)
work['address'] = work.apply(lambda row: preprocess_address(row.address), axis = 1)

In [7]:
from difflib import SequenceMatcher
def similar(a, b):
    a = str(a).upper().strip()
    b = str(b).upper().strip()
    rp_a = ''
    rp_b = ''
    for s in a.split(' '):
        rp_a += s
    for s in b.split(' '):
        rp_b += s
    return SequenceMatcher(None, rp_a, rp_b).ratio()
## province dict
province_dict = {
    "An Giang": 1,
    "Kon Tum": 33,
"Bà Rịa – Vũng Tàu": 2,
"Lai Châu": 34,
"Bắc Giang"	: 3,
"Lâm Đồng": 35,
	"Bắc Kạn": 4,	
	"Lạng Sơn": 36,
	"Bạc Liêu": 5,	
	"Lào Cai": 37,
	"Bắc Ninh": 6, 	
	"Long An": 38,
	"Bến Tre": 7,	
	"Nam Định": 39,
	"Bình Định"	: 8,
	"Nghệ An": 40,
	"Bình Dương": 9	,
	"Ninh Bình": 41,
	"Bình Phước"	: 10,
	"Ninh Thuận": 42,
	"Bình Thuận"	: 11,
	"Phú Thọ": 43,
	"Cà Mau"	: 12,
	"Phú Yên": 44,
	"Cần Thơ"	: 13,
	"Quảng Bình": 45,
	"Cao Bằng"	: 14,
	"Quảng Nam" : 46,
	"Đà Nẵng" : 15,
	"Quảng Ngãi": 47,
	"Đắk Lắk"	: 16,
	"Quảng Ninh" : 48, 
	"Đắk Nông"	: 17,
	"Quảng Trị": 49,
	"Điện Biên"	: 18,
	"Sóc Trăng": 50,
	"Đồng Nai"	: 19,
	"Sơn La": 51,
	"Đồng Tháp"	: 20,
	"Tây Ninh": 52,
	"Gia Lai"	: 21,
	"Thái Bình": 53,
	"Hà Giang"	: 22,
	"Thái Nguyên": 54,
	"Hà Nam"	: 23,
	"Thanh Hóa": 55,
	"Hà Nội"	: 24,
	"Thừa Thiên Huế": 56,
	"Hà Tĩnh"	: 25,
	"Tiền Giang": 57,
	"Hải Dương"	: 26,
	"TP Hồ Chí Minh" : 58,
	"Hải Phòng"	: 27, 
	"Trà Vinh": 59,
	"Hậu Giang"	: 28,
	"Tuyên Quang": 60, 
	"Hòa Bình"	: 29,
	"Vĩnh Long": 61,
	"Hưng Yên"	: 30,
	"Vĩnh Phúc": 62,
	"Khánh Hòa"	: 31,
	"Yên Bái": 63,
	"Kiên Giang": 32	 	 
}

In [8]:
## caculate similarity
for k, v in province_dict.items():
    col_name = str(v)
    work[col_name] = work.apply(lambda row: similar( row.address, k), axis = 1)

In [9]:
## caculate max similarity
province_id = list(range(1,64))
province_col_name  = [str(x) for x in province_id]
work['max_similar_score'] = work[province_col_name].max(axis= 1)
work['work_province_id'] = work[province_col_name].idxmax(axis= 1)

## thresh hold = 0.5
work['new_work_province_id']  = work.apply(lambda row: row.work_province_id if row.max_similar_score > 0.5 else -999, axis = 1)

## 2. employee_lv

In [10]:
## outlier : -1, 331, 369 - Nan dien la -1
work['employee_lv'] = work['employee_lv'].fillna(-1)
work['employee_lv'] = work['employee_lv'].apply(lambda x: x if(x < 300) else -1 )
work['employee_lv'].describe()

count    409842.000000
mean          8.089908
std           8.250423
min        -999.000000
25%           5.000000
50%           7.000000
75%           9.000000
max          59.000000
Name: employee_lv, dtype: float64

## 3. date DA CO ROI

## 4. job/role

## 5. company_type

In [11]:
## k co null
work['company_type'] = work['company_type'].astype('category')

## 6. id_management	

## 7. id_office	

In [12]:
use_cols = [
    'id',
    'id_bh',
    'new_work_province_id',
    'employee_lv',
    'from_date','to_date','year_from_date', 'year_to_date', 
    'month_from_date', 'month_to_date', 'num_year_contract', 'num_month_contract',
    "job/role", 
    "company_type",
    "id_management",
    "id_office"
] 

In [13]:
export_work = work[use_cols]
export_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409842 entries, 0 to 162282
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   id                    409842 non-null  int64   
 1   id_bh                 409842 non-null  int64   
 2   new_work_province_id  409842 non-null  object  
 3   employee_lv           409842 non-null  float64 
 4   from_date             409842 non-null  int64   
 5   to_date               409842 non-null  int64   
 6   year_from_date        409842 non-null  int64   
 7   year_to_date          409842 non-null  int64   
 8   month_from_date       409842 non-null  int64   
 9   month_to_date         409842 non-null  int64   
 10  num_year_contract     409842 non-null  float64 
 11  num_month_contract    409842 non-null  int64   
 12  job/role              409832 non-null  object  
 13  company_type          409842 non-null  category
 14  id_management         409842 non-nul

In [14]:
## cluster job/role

In [15]:
## cluster job role
from sklearn.feature_extraction.text import TfidfVectorizer
export_work['job_role_fillNan'] = export_work['job/role'].fillna("none")
vectorizer = TfidfVectorizer(
    ngram_range = (1,2),
        min_df=5, 
    max_df= 0.8, 
    max_features=1000,
    sublinear_tf=True
)
job_role = export_work['job_role_fillNan'].values
X_job_role = vectorizer.fit_transform(job_role)

C:\Users\Toan\AppData\Local\Temp/ipykernel_20472/453157816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_work['job_role_fillNan'] = export_work['job/role'].fillna("none")


In [16]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

In [18]:
kmeanModel = KMeans(n_clusters=300, random_state = 2022).fit(X_job_role) 
kmeanModel.labels_

array([ 24,  24, 134, ..., 130,  49,  49])

In [19]:
export_work['job_role_encode_knn'] = kmeanModel.labels_

C:\Users\Toan\AppData\Local\Temp/ipykernel_20472/3463333387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_work['job_role_encode_knn'] = kmeanModel.labels_


In [20]:
train_export_work = export_work[:len(train_work)]
test_export_work = export_work[len(train_work):]

In [23]:
train_export_work.columns

Index(['id', 'id_bh', 'new_work_province_id', 'employee_lv', 'from_date',
       'to_date', 'year_from_date', 'year_to_date', 'month_from_date',
       'month_to_date', 'num_year_contract', 'num_month_contract', 'job/role',
       'company_type', 'id_management', 'id_office', 'job_role_fillNan',
       'job_role_encode_knn'],
      dtype='object')

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
vectorizer = TfidfVectorizer(
    ngram_range = (1,2),
        min_df=5, 
    max_df= 0.8, 
    max_features=1000,
    sublinear_tf=True
)
## TF-IDF + SVD
clf = Pipeline([
                ('tfidf', vectorizer ),
                ('svd', TruncatedSVD(n_components = 128, random_state=42)),
                ])
clf.fit(train_export_work['job_role_fillNan'].values)
train_job = clf.transform(train_export_work['job_role_fillNan'].values)
test_job = clf.transform(test_export_work['job_role_fillNan'].values)

In [29]:
##  CONCAT 
x = np.concatenate((train_job, test_job), axis=0)
x.shape

(409842, 128)

In [30]:
kmeanModel_2 = KMeans(n_clusters=300, random_state = 2022).fit(x) 
kmeanModel_2.labels_

array([  8,   8,  70, ..., 231, 170, 170])

In [31]:
export_work['job_role_encode_knn_ver2_svd'] = kmeanModel_2.labels_

C:\Users\Toan\AppData\Local\Temp/ipykernel_20472/791237441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_work['job_role_encode_knn_ver2_svd'] = kmeanModel_2.labels_


In [32]:
train_export_work = export_work[:len(train_work)]
test_export_work = export_work[len(train_work):]

In [33]:
##  encoder cho : id_office , id_management     --> sau do Fill nan -999  
from sklearn.preprocessing import  OrdinalEncoder   
cols = ['id_office']
## replace -999 = Nan
train_export_work[cols].replace( -999, np.nan)
test_export_work[cols].replace( -999, np.nan)
train_export_work[cols].replace( '-999', np.nan)
test_export_work[cols].replace( '-999', np.nan)
##
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-999)
train_export_work[cols] = encoder.fit_transform(train_export_work[cols].to_numpy().reshape(-1, 1))
test_export_work[cols] = encoder.transform(test_export_work[cols].to_numpy().reshape(-1, 1))  

C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [34]:
cols = ['id_management']
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-999)
train_export_work[cols] = encoder.fit_transform(train_export_work[cols].to_numpy().reshape(-1, 1))
test_export_work[cols] = encoder.transform(test_export_work[cols].to_numpy().reshape(-1, 1))  

C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [35]:
## fill nan -999 id_office             
train_export_work['id_office'] = train_export_work['id_office'].fillna(-999)
test_export_work['id_office'] = test_export_work['id_office'].fillna(-999)

C:\Users\Toan\AppData\Local\Temp/ipykernel_20472/27997092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_export_work['id_office'] = train_export_work['id_office'].fillna(-999)
C:\Users\Toan\AppData\Local\Temp/ipykernel_20472/27997092.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_export_work['id_office'] = test_export_work['id_office'].fillna(-999)


In [36]:
train_export_work.to_csv(r"data/dat_fill_work_data/processed_train_work.csv")
test_export_work.to_csv(r"data/dat_fill_work_data/processed_test_work.csv")